In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.streaming import DataStreamReader


# conf = pyspark.SparkConf().setAll([('spark.executor.memory', '1g'), ('spark.executor.cores', '1'), ('spark.cores.max', '1'), ('spark.driver.memory','2g')])

spark = (SparkSession.builder
         .master("local")
         .appName("Transactions")
         .config('spark.executor.memory', '1g')
         .config('spark.executor.cores', '1')
         .config('spark.driver.memory','2g')
         .getOrCreate()
        )
person_read = DataStreamReader(spark)
trans_read = DataStreamReader(spark)
sent_read = DataStreamReader(spark)
received_read = DataStreamReader(spark)
merchant_read = DataStreamReader(spark)
loc_read = DataStreamReader(spark)
acct_read = DataStreamReader(spark)
cc_read = DataStreamReader(spark)
resides_read = DataStreamReader(spark)

spark.version
# pw = "zeppelin"

24/04/21 11:42:04 WARN Utils: Your hostname, isaac-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.106.130 instead (on interface ens33)
24/04/21 11:42:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/21 11:42:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.5.1'

In [2]:
schema = spark.read.csv("Sparkov_Data_Generation/output/adults_2550_male_urban_00-327.csv", sep="|", header=True).schema
# schema

In [4]:
path = "Sparkov_Data_Generation/output2/"

class Stream:
    def __init__(self, spark_read, path, schema):
        self.read_stream = spark_read.csv(path=path, sep="|", schema=schema)

    def insert_nodes(self, fields, checkpoint_path, label, key):
        self.read_stream.select(fields).dropna(how="any").writeStream \
          .format("org.neo4j.spark.DataSource") \
          .option("url", "bolt://localhost:7687") \
          .option("checkpointLocation", checkpoint_path) \
          .option("labels", label) \
          .option("node.keys", key) \
          .option("SaveMode", "Overwrite") \
          .start()

    def insert_relationships(self, fields, checkpoint_path, relationship, labels_keys):
        self.read_stream.select(fields).dropna(how="any").writeStream \
          .format("org.neo4j.spark.DataSource") \
          .option("relationship", relationship) \
          .option("url", "bolt://localhost:7687") \
          .option("checkpointLocation", checkpoint_path) \
          .option("relationship.save.strategy", "keys") \
          .option("relationship.source.labels", labels_keys.get("s_label")) \
          .option("relationship.source.save.mode", "Overwrite") \
          .option("relationship.source.node.keys", labels_keys.get("s_key")) \
          .option("relationship.target.labels", labels_keys.get("t_label")) \
          .option("relationship.target.save.mode", "Overwrite") \
          .option("relationship.target.node.keys", labels_keys.get("t_key")) \
          .start()

per_stream = Stream(person_read, path, schema)
tran_stream = Stream(trans_read, path, schema)
sent_stream = Stream(sent_read, path, schema)
received_stream = Stream(received_read, path, schema)
merchant_stream = Stream(merchant_read, path, schema)
loc_stream = Stream(loc_read, path, schema)
acct_stream = Stream(acct_read, path, schema)
cc_stream = Stream(cc_read, path, schema)
resides_stream = Stream(resides_read, path, schema)

In [5]:
person_fields = ["ssn", "first", "last", "gender", "job", "dob"]
acct_fields = ["acct_num"]
cc_fields = ["cc_num"]
per_loc_fields = ["street","city","state","zip","lat","long","city_pop"]
mer_loc_fields = ["merch_lat","merch_long"]
trans_fields = ["trans_num","trans_date","trans_time","amt"]
merch_fields = ["category","merchant"]

In [6]:
per_stream.insert_nodes(person_fields, "/tmp/chpt1", ":Person", "ssn")
tran_stream.insert_nodes(trans_fields, "/tmp/chpt2", ":Transaction", "trans_num")
merchant_stream.insert_nodes(merch_fields, "/tmp/chpt3", ":Merchant", "merchant")
loc_stream.insert_nodes(per_loc_fields, "/tmp/chpt4", ":Location", "city")
acct_stream.insert_nodes(acct_fields, "/tmp/chpt5", ":Account", "acct_num")
cc_stream.insert_nodes(cc_fields, "/tmp/chpt6", ":CreditCard", "cc_num")


sent_stream.insert_relationships(["ssn","trans_num"],
                                "/tmp/chpt7",
                                "SENT",
                                {
                                    "s_label": ":Person",
                                    "s_key": "ssn",
                                    "t_label":":Transaction",
                                    "t_key": "trans_num"
                                })
received_stream.insert_relationships(["trans_num", "merchant"],
                                "/tmp/chpt8",
                                "RECEIVED",
                                {
                                    "s_label": ":Transaction",
                                    "s_key": "trans_num",
                                    "t_label":":Merchant",
                                    "t_key": "merchant"
                                })
resides_stream.insert_relationships(["ssn","city"],
                                "/tmp/chpt9",
                                "RESIDES_IN",
                                {
                                    "s_label": ":Person",
                                    "s_key": "ssn",
                                    "t_label":":Location",
                                    "t_key": "city"
                                })

24/04/21 11:43:19 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/21 11:43:21 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/21 11:43:22 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/21 11:43:22 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/21 11:43:23 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/21 11:43:23 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/21 11:43:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be di

In [8]:
# # transactions = stream.select("trans_num","trans_date","trans_time","amt")

# per_stream.select(person_fields).writeStream \
#   .format("org.neo4j.spark.DataSource") \
#   .option("url", "bolt://localhost:7687") \
#   .option("checkpointLocation", "/tmp/chpt/") \
#   .option("labels", ":Person") \
#   .option("node.keys", "ssn") \
#   .option("SaveMode", "Overwrite") \
#   .start()

# tran_stream.select(trans_fields).dropna(how="any",subset="trans_num").writeStream \
#   .format("org.neo4j.spark.DataSource") \
#   .option("url", "bolt://localhost:7687") \
#   .option("checkpointLocation", "/tmp/chpt2/") \
#   .option("labels", ":Transaction") \
#   .option("node.keys", "trans_num") \
#   .option("SaveMode", "Overwrite") \
#   .start()

# merchant_stream.select(merch_fields).writeStream \
#   .format("org.neo4j.spark.DataSource") \
#   .option("url", "bolt://localhost:7687") \
#   .option("checkpointLocation", "/tmp/chpt4/") \
#   .option("labels", ":Merchant") \
#   .option("node.keys", "merchant") \
#   .option("SaveMode", "Overwrite") \
#   .start()

24/04/21 09:14:20 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/21 09:14:21 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/21 09:14:22 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [9]:
# # relationship streams
# sent_stream.select("ssn","trans_num").dropna(how="any",subset="trans_num").writeStream \
#   .format("org.neo4j.spark.DataSource") \
#   .option("relationship","SENT") \
#   .option("url", "bolt://localhost:7687") \
#   .option("checkpointLocation", "/tmp/chpt3/") \
#   .option("relationship.save.strategy", "keys") \
#   .option("relationship.source.labels", "Person") \
#   .option("relationship.source.save.mode", "Overwrite") \
#   .option("relationship.source.node.keys", "ssn") \
#   .option("relationship.target.labels", "Transaction") \
#   .option("relationship.target.save.mode", "Overwrite") \
#   .option("relationship.target.node.keys", "trans_num") \
#   .start()

# received_stream.select("merchant","trans_num").dropna(how="any",subset="trans_num").writeStream \
#   .format("org.neo4j.spark.DataSource") \
#   .option("relationship","RECEIVED") \
#   .option("url", "bolt://localhost:7687") \
#   .option("checkpointLocation", "/tmp/chpt5/") \
#   .option("relationship.save.strategy", "keys") \
#   .option("relationship.source.labels", "Transaction") \
#   .option("relationship.source.save.mode", "Overwrite") \
#   .option("relationship.source.node.keys", "trans_num") \
#   .option("relationship.target.labels", "Merchant") \
#   .option("relationship.target.save.mode", "Overwrite") \
#   .option("relationship.target.node.keys", "merchant") \
#   .start()

24/04/21 09:14:38 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/21 09:14:39 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/04/21 09:34:30 WARN FileStreamSource: Listed 2 file(s) in 10036 ms           
24/04/21 09:34:30 WARN FileStreamSource: Listed 2 file(s) in 9990 ms
24/04/21 09:34:30 WARN FileStreamSource: Listed 2 file(s) in 8357 ms
24/04/21 09:34:30 WARN FileStreamSource: Listed 2 file(s) in 8357 ms


In [39]:
# spark.stop()